In [ ]:
from flask import Flask, render_template, request
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


ratings = pd.read_csv('/Users/siyaksares/Developer/GitHub/ReadersRadarProject/ratings.csv')

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)

trainset = data.build_full_trainset()
svd_model = SVD()
svd_model.fit(trainset)

user_item_matrix = ratings.pivot(index='user_id', columns='book_id', values='rating')
user_item_matrix.fillna(0, inplace=True)

In [ ]:
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_items, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.fc = nn.Linear(embedding_size, 1)

    def forward(self, user, item):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        x = user_embedding * item_embedding
        x = self.fc(x)
        return x


num_users = ratings['user_id'].nunique()
num_items = ratings['book_id'].nunique()
model = RecommenderNet(num_users, num_items)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


user_ids = torch.tensor(ratings['user_id'].values, dtype=torch.long)
item_ids = torch.tensor(ratings['book_id'].values, dtype=torch.long)
ratings_values = torch.tensor(ratings['rating'].values, dtype=torch.float)


for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(user_ids, item_ids).squeeze()
    loss = criterion(outputs, ratings_values)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np

app = Flask(__name__)

# Load data
tags = pd.read_csv('/path/to/tags.csv')
book_tags = pd.read_csv('/path/to/book_tags.csv')
books = pd.read_csv('/path/to/books.csv')
ratings = pd.read_csv('/path/to/ratings.csv')

# Initialize models (Collaborative filtering and neural network)
# (Include the above-trained models here)

def normalize_tag_name(tag_name):
    if pd.isnull(tag_name):
        return ''
    return tag_name.lower().replace(r'[^a-zA-Z0-9\s]', '')

tags['tag_name_normalized'] = tags['tag_name'].apply(normalize_tag_name)

valid_tag_ids = tags['tag_id'].unique()
book_tags = book_tags[book_tags['tag_id'].isin(valid_tag_ids)]

def filter_books_by_tag(book_tags, tags, selected_tag):
    selected_tag_normalized = normalize_tag_name(selected_tag)
    selected_tag_id = tags.loc[tags['tag_name_normalized'] == selected_tag_normalized, 'tag_id'].values
    if len(selected_tag_id) == 0:
        return pd.DataFrame()
    filtered_books = book_tags.merge(tags, on='tag_id')
    filtered_books = filtered_books[filtered_books['tag_name_normalized'] == selected_tag_normalized]
    return filtered_books

def recommend_book(user_books, books, model):
    if len(user_books) == 0:
        return {}

    user_tensor = torch.tensor([1] * len(user_books), dtype=torch.long)
    item_tensor = torch.tensor(user_books, dtype=torch.long)
    model.eval()
    with torch.no_grad():
        predictions = model(user_tensor, item_tensor).squeeze()
    top_item = predictions.argmax().item()
    recommended_book = books.loc[books['book_id'] == top_item].to_dict(orient='records')[0]
    return recommended_book

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommend', methods=['GET', 'POST'])
def recommend():
    if request.method == 'GET':
        top_tag_ids = book_tags['tag_id'].value_counts().head(10).index
        top_tag_names = tags[tags['tag_id'].isin(top_tag_ids)]['tag_name']
        return render_template('recommend_form.html', top_tags=top_tag_names)
    elif request.method == 'POST':
        user_books = request.form.getlist('user_books')
        user_books = [int(book) for book in user_books]
        recommended_book = recommend_book(user_books, books, model)
        return render_template('result.html', recommended_book=recommended_book)

if __name__ == '__main__':
    app.run(debug=True, port=3000)
